# Wiener's Attack on RSA

## Introduction

This notebook provides a complete implementation of the Wiener attack on RSA, a cryptanalytic technique that targets RSA keys with small private exponents. The attack was first discovered by Michael J. Wiener in 1990 and exploits the properties of continued fractions to efficiently recover the private key when certain conditions are met.

RSA is one of the most widely used public key cryptosystems, but it can become vulnerable if its parameters are not chosen correctly. Specifically, if the private exponent d is too small relative to the modulus N, the Wiener attack can break the encryption in polynomial time.

## Mathematical Background

### RSA Cryptosystem Basics

In RSA:
- We have a modulus $N = p \times q$ where $p$ and $q$ are large primes
- Public exponent $e$ and private exponent $d$ satisfy $e \times d \equiv 1 \pmod{\phi(N)}$
- $\phi(N) = (p-1) \times (q-1)$ is Euler's totient function
- Public key is $(e, N)$, private key is $(d, N)$

### Wiener's Attack Theory

Wiener discovered that if $d < \frac{1}{3} \times N^{1/4}$, then we can efficiently find $d$ using continued fraction expansion of $\frac{e}{N}$.

The attack is based on the fact that:
- $e \times d - k \times \phi(N) = 1$ for some integer $k$
- If $d$ is small, then $\frac{k}{d}$ is a close approximation to $\frac{e}{\phi(N)}$
- $\frac{e}{N}$ is a close approximation to $\frac{e}{\phi(N)}$
- Therefore, $\frac{k}{d}$ is one of the convergents in the continued fraction expansion of $\frac{e}{N}$

## 1. Helper Functions

The first cell defines helper functions: 
- one to compute the continued fraction expansion for a rational number
- one to compute the convergents from that expansion
- one to check whether a number is a perfect square.

In [24]:
import math

def continued_fraction_expansion(numerator, denominator):
    """
    Computes the continued fraction expansion of a rational number numerator/denominator.
    
    Returns:
        A list of integers representing the coefficients in the continued fraction.
    """
    cf = []
    while denominator:
        a = numerator // denominator
        cf.append(a)
        numerator, denominator = denominator, numerator - a * denominator
    return cf

def convergents_from_cf(cf):
    """
    Given the continued fraction coefficients (cf), compute the convergents.
    
    Returns:
        A list of tuples (numerator, denominator) representing each convergent.
    """
    convergents = []
    for i in range(len(cf)):
        if i == 0:
            convergents.append((cf[0], 1))
        elif i == 1:
            h = cf[0]*cf[1] + 1
            k = cf[1]
            convergents.append((h, k))
        else:
            h1, k1 = convergents[i-1]
            h2, k2 = convergents[i-2]
            h = cf[i]*h1 + h2
            k = cf[i]*k1 + k2
            convergents.append((h, k))
    return convergents

def is_perfect_square(n):
    """
    Checks if n is a perfect square. 
    
    Returns:
        The integer square root of n if n is a perfect square, or None otherwise.
    """
    if n < 0:
        return None
    root = int(math.isqrt(n))
    return root if root*root == n else None

### Explanation:

- `continued_fraction_expansion` computes the expansion by repeatedly applying Euclid's algorithm.
- `convergents_from_cf` builds the sequence of convergents from the continued fraction coefficients.
- `is_perfect_square` is used to check whether the discriminant we compute later is a perfect square.

## 2. Implementing Wiener's Attack

The following function attempts to recover the private exponent d from the public key (e, n).

In [25]:
def wiener_attack(e, n):
    """
    Performs Wiener's attack on an RSA public key (e, n).
    
    If successful, returns the private exponent d. If the attack fails (i.e. if d is not small
    enough or the computed candidates do not yield a valid factorization of n), returns None.
    """
    # Compute the continued fraction expansion for e/n.
    cf = continued_fraction_expansion(e, n)
    # Generate the convergents from the continued fraction.
    convs = convergents_from_cf(cf)
    
    for (k, d_candidate) in convs:
        # Candidate d must be positive and k must be non-zero.
        if k == 0:
            continue
        
        # Check if (e*d_candidate - 1) is divisible by k.
        if (e * d_candidate - 1) % k != 0:
            continue
        
        # Compute the possible phi(n)
        phi_candidate = (e * d_candidate - 1) // k
        
        # Now, using the relation phi(n) = n - p - q + 1, we can set up the quadratic:
        #   x^2 - (n - phi(n) + 1)*x + n = 0
        # where x represents one of the prime factors.
        a = 1
        b = -(n - phi_candidate + 1)
        c = n
        
        discriminant = b * b - 4 * a * c
        sqrt_discriminant = is_perfect_square(discriminant)
        if sqrt_discriminant is None:
            continue
        
        # Compute the roots of the quadratic equation.
        x1 = (-b + sqrt_discriminant) // 2
        x2 = (-b - sqrt_discriminant) // 2
        
        # Check if the found roots multiply to n.
        if x1 * x2 == n:
            # If they do, then d_candidate is the private exponent.
            return d_candidate
    return None

### Explanation:

For each convergent `(k, d_candidate)` of the continued fraction expansion of e/n, the code:

1. Checks if `(e · d_candidate – 1)` is divisible by k.

2. If so, computes a candidate for φ(n) using:
   φ_candidate = (e·d – 1)⁄k

3. Then, it sets up the quadratic equation derived from the relation:
   φ(n) = (p – 1)(q – 1) = n – (p + q) + 1

4. It checks if the discriminant is a perfect square. If the factors computed from the quadratic multiply to n, then the candidate d is correct.

## 3. Interactive Input and Testing

This cell enables you to input your RSA public key parameters and see the result. (For demonstration, if you have parameters where the private exponent is small, the attack will recover it.)

In [ ]:
def main():
    print("Wiener's Attack on RSA")
    print("----------------------")
    try:
        # For interactive input, ensure you provide integer values.
        e = int(input("Enter the RSA public exponent (e): "))
        n = int(input("Enter the RSA modulus (n): "))
    except ValueError:
        print("Invalid input. Ensure you enter integers for e and n.")
        return
    
    d = wiener_attack(e, n)
    
    if d is not None:
        print("\nSuccess! The private exponent d is:")
        print(d)
    else:
        print("\nWiener's attack failed. It seems the private exponent is not small enough.")
        
if __name__ == "__main__":
    main()

Wiener's Attack on RSA
----------------------


### Explanation:

- The `main()` function provides a simple interactive prompt where you can supply your RSA public key values.
- If the attack is successful, it prints the recovered private exponent d; if not, it notifies you that the attack did not work (which most likely means d isn't small enough).

## 4. Additional Explanation and Testing Ideas

### Why Does the Attack Work?
The success of Wiener's attack is rooted in the theory of continued fractions. When d is very small (compared to n), the fraction e/n (or more precisely, e/φ(n)) is well approximated by one of its convergents. This convergent gives us a candidate fraction k/d that satisfies the equation ed – k·φ(n) = 1, enabling the recovery of φ(n) and eventually the factors of n.

### Testing the Notebook:
To successfully test this notebook, use an RSA key pair known to have a small private exponent. In real-life RSA implementations, using a small d is strongly discouraged because it makes the key vulnerable to Wiener's attack.

### Further Extensions:
You might consider extending the notebook to:

1. Generate RSA keys with small d to demonstrate the vulnerability.
2. Visualize the continued fraction convergents and track the progress of the algorithm.
3. Compare the running time with various key sizes to see how the vulnerability scales.

## 5. Example: Testing with a Vulnerable RSA Key

Let's create a function to generate an RSA key pair with a deliberately small private exponent for testing purposes.

In [20]:
import random
from math import gcd

def generate_vulnerable_rsa(bits=512, d_bits=100):
    """
    Generate an RSA key pair with a deliberately small private exponent for testing.
    
    Args:
        bits: The bit length of the RSA modulus n.
        d_bits: The bit length of the private exponent d.
        
    Returns:
        A tuple (e, n, d, p, q) where (e, n) is the public key, d is the private key,
        and p, q are the prime factors of n.
    """
    # Generate two random prime numbers
    from sympy import randprime
    p = randprime(2**(bits//2 - 1), 2**(bits//2))
    q = randprime(2**(bits//2 - 1), 2**(bits//2))
    
    n = p * q
    phi = (p - 1) * (q - 1)
    
    # Generate a small value for d
    d = randprime(2**(d_bits-1), 2**d_bits)
    
    # Ensure d and phi are coprime
    while gcd(d, phi) != 1:
        d = randprime(2**(d_bits-1), 2**d_bits)
    
    # Calculate e such that e*d ≡ 1 (mod phi)
    e = pow(d, -1, phi)
    
    return (e, n, d, p, q)

# Note: This requires the sympy library. If not installed, use: pip install sympy

In [22]:
try:
    # Try to generate a vulnerable RSA key pair
    e, n, actual_d, p, q = generate_vulnerable_rsa()
    
    print(f"Generated vulnerable RSA key pair:")
    print(f"Public exponent (e): {e}")
    print(f"Modulus (n): {n}")
    print(f"Actual private exponent (d): {actual_d}")
    print(f"Prime factors: p={p}, q={q}")
    
    # Test the Wiener's attack
    print("\nTesting Wiener's attack...")
    recovered_d = wiener_attack(e, n)
    
    if recovered_d is not None:
        print(f"Attack successful! Recovered private exponent: {recovered_d}")
        print(f"Is the recovered d correct? {recovered_d == actual_d}")
    else:
        print("Attack failed. The private exponent might not be small enough for Wiener's attack.")
except ImportError:
    print("This example requires the sympy library. Install it using: pip install sympy")
except Exception as ex:
    print(f"An error occurred: {ex}")

Generated vulnerable RSA key pair:
Public exponent (e): 5062935746356527563081865770588775843308752772654321623779020808404651421376353375502789815210019421229690860448230864570146154294912528854524084456650879
Modulus (n): 9039712506836259983464415995539462219446775874774939727198743423918076293272804475581328092229151831563822903110518819693790780148288253097262851857756563
Actual private exponent (d): 927650960267079454366656486719
Prime factors: p=94418012898455753022355112587502689951827297889526901623670086167409216751313, q=95741397529285519916237432812584879769770269148161114868711585521493242729251

Testing Wiener's attack...
Attack successful! Recovered private exponent: 927650960267079454366656486719
Is the recovered d correct? True


## References

1. Wiener, M. J. (1990). "Cryptanalysis of short RSA secret exponents". IEEE Transactions on Information Theory, 36(3), 553-558.
2. Boneh, D., & Durfee, G. (2000). "Cryptanalysis of RSA with private key d less than N^0.292". IEEE Transactions on Information Theory, 46(4), 1339-1349.
3. Dujella, A. (2004). "Continued fractions and RSA with small secret exponent". Tatra Mountains Mathematical Publications, 29(1), 101-112.
4. Howgrave-Graham, N. (1999). "Finding small roots of univariate modular equations revisited". In Crytography and Coding (pp. 131-142). Springer Berlin Heidelberg.